In [1]:
import findspark
import os

In [2]:
SPARK_HOME = os.environ.get('SPARK_HOME')
SPARK_HOME

'/opt/spark'

In [3]:
findspark.init(spark_home=SPARK_HOME)

In [4]:
import pyspark
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/03/06 14:58:35 WARN Utils: Your hostname, DESKTOP-TINF367 resolves to a loopback address: 127.0.1.1; using 172.28.17.111 instead (on interface eth0)
24/03/06 14:58:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/06 14:58:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
# Read zipped file directly from Spark
df_zipped = spark \
    .read \
    .format("csv") \
    .option("header", True) \
    .load("fhv_tripdata_2019-10.csv")
df_zipped.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   null|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   null|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   null|                B00014|
|     B00021         |2019-10-01 00:00:4

In [7]:
!head -n 1001 fhv_tripdata_2019-10.csv > headlinall.csv

In [9]:
import pandas as pd

In [10]:
second_df = pd.read_csv('headlinall.csv')

second_df.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID              float64
DOlocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [13]:
spark.createDataFrame(second_df).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [17]:
from pyspark.sql import types

In [18]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropOff_datetime', types.TimestampType(), True), 
    types.StructField('PUlocationID', types.IntegerType(), True), 
    types.StructField('DOlocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True), 
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [19]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv("fhv_tripdata_2019-10.csv")

In [20]:
# partition
df = df.repartition(6)

In [21]:
# write to parquet

df.write.parquet('fhv/2019/10/')

In [22]:
# How many taxi trips were there on the 15th of October?

# spark dataframes
df = spark.read.parquet('fhv/2019/10/')
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [26]:
df_fifteenth_of_october = df.select('dispatching_base_num', 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
    .filter((df.pickup_datetime >= '2019-10-15') & (df.pickup_datetime < '2019-10-16')) \
    .count()
df_fifteenth_of_october

62610

In [27]:
# Longest trip for each day

# What is the length of the longest trip in the dataset in hours?

# Get started by creating temp tables <====>
df.registerTempTable("trips_data")

/opt/spark/python/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [28]:
spark.sql("""
SELECT
    pickup_datetime,
    dropoff_datetime,
    PULocationID,
    DOLocationID,
    TIMESTAMPDIFF(HOUR, pickup_datetime, dropoff_datetime) AS difference

FROM
    trips_data
ORDER BY
    difference DESC
""").show()

+-------------------+-------------------+------------+------------+----------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|difference|
+-------------------+-------------------+------------+------------+----------+
|2019-10-28 09:00:00|2091-10-28 09:30:00|         264|         264|    631152|
|2019-10-11 18:00:00|2091-10-11 18:30:00|         264|         264|    631152|
|2019-10-31 23:46:33|2029-11-01 00:13:00|        null|        null|     87672|
|2019-10-01 21:43:42|2027-10-01 21:45:23|         159|         264|     70128|
|2019-10-17 14:00:00|2020-10-18 00:00:00|        null|        null|      8794|
|2019-10-26 21:26:00|2020-10-26 21:36:00|         264|         264|      8784|
|2019-10-30 12:30:04|2019-12-30 13:02:08|         264|          50|      1464|
|2019-10-25 07:04:57|2019-12-08 07:21:11|         168|         235|      1056|
|2019-10-25 07:04:57|2019-12-08 07:54:33|         168|         235|      1056|
|2019-10-01 07:21:12|2019-11-03 08:44:21|           

In [29]:
# Least frequent pickup location zone

# Load the zone lookup data into a temp view in Spark

zone_df = spark.read.option("header", "true").csv('taxi_zone_lookup.csv')

zone_df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [36]:
# zone_df.registerTempTable("zone_data")

# join zone_df to main df

df_join = df.join(zone_df, df.PUlocationID == zone_df.LocationID, 'inner')

# zone_df.show()

In [37]:
# create table from the joined table

df_join.registerTempTable("join_data")
df_join.show()

/opt/spark/python/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+---------+--------------------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|LocationID|  Borough|                Zone|service_zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+---------+--------------------+------------+
|              B00445|2019-10-07 03:54:41|2019-10-07 04:02:41|         252|         138|   null|                B00445|       252|   Queens|          Whitestone|   Boro Zone|
|              B02613|2019-10-05 00:47:05|2019-10-05 01:11:53|         264|          39|   null|                B02613|       264|  Unknown|                  NV|         N/A|
|              B01437|2019-10-02 20:29:31|2019-10-02 20:52:31|         264|         173|   null|                B01437|      

In [38]:
spark.sql("""
SELECT
    Zone,
    COUNT(Zone) AS Zone_Frequency
FROM
    join_data
GROUP BY Zone
ORDER BY Zone_Frequency
""").show()

+--------------------+--------------+
|                Zone|Zone_Frequency|
+--------------------+--------------+
|         Jamaica Bay|             1|
|Governor's Island...|             2|
| Green-Wood Cemetery|             5|
|       Broad Channel|             8|
|     Highbridge Park|            14|
|        Battery Park|            15|
|Saint Michaels Ce...|            23|
|Breezy Point/Fort...|            25|
|Marine Park/Floyd...|            26|
|        Astoria Park|            29|
|    Inwood Hill Park|            39|
|       Willets Point|            47|
|Forest Park/Highl...|            53|
|  Brooklyn Navy Yard|            57|
|        Crotona Park|            62|
|        Country Club|            77|
|     Freshkills Park|            89|
|       Prospect Park|            98|
|     Columbia Street|           105|
|  South Williamsburg|           110|
+--------------------+--------------+
only showing top 20 rows

